<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Machine Learning with PyTorch for Developers</h1>
<h1>Machine Learning Overview</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [ ]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import torch
from torch import nn
import torch.nn.functional as F

import watermark

%load_ext watermark
%matplotlib inline

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


We start by print out the versions of the libraries we're using for future reference

In [6]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 9.0.0

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 46fd989832f97305d47c167037aa491a1802660f

torch     : 2.6.0
matplotlib: 3.10.1
numpy     : 2.2.3
watermark : 2.5.0
pandas    : 2.2.3



Load default figure style

In [7]:
plt.style.use('./d4sci.mplstyle')

# Start

In [8]:
"""
Below is an illustrative overview of PyTorch fundamentals, focusing on torch.nn and
torch.nn.functional for defining and implementing optimization problems.

1. Tensor Basics
----------------
PyTorch tensors are array-like structures that can run on CPU or GPU. They support
autograd, meaning gradients are automatically calculated for operations on tensors
marked with requires_grad=True.

2. torch.nn
-----------
• Provides building blocks for neural networks (e.g., Linear, Conv2d).
• Encapsulates parameters (weights/biases) that are automatically tracked.

3. torch.nn.functional
----------------------
• Contains functional forms of common layers and activation functions (e.g., F.relu, F.sigmoid).
• Often used inside the forward() method without creating explicit layer objects.

4. Defining a Model
-------------------
• Inherit from nn.Module.
• Define the layers in __init__.
• Specify forward pass in forward().

5. Optimization
---------------
• Use built-in optimizers (e.g., torch.optim.SGD, torch.optim.Adam). 
• Typically define a loss function (e.g., nn.MSELoss, nn.CrossEntropyLoss) and call
    loss.backward() to compute gradients, then optimizer.step() to update parameters.

--------------------------------------------------------
Example: Simple feed-forward network with a training loop
--------------------------------------------------------
"""

# Simple feed-forward neural network with one hidden layer
class SimpleFeedForwardNN(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
                super(SimpleFeedForwardNN, self).__init__()
                self.fc1 = nn.Linear(input_size, hidden_size)
                self.fc2 = nn.Linear(hidden_size, output_size)

        def forward(self, x):
                # Use torch.nn.functional for activation
                x = F.relu(self.fc1(x))
                return self.fc2(x)

# Instantiate the model, define loss and optimizer
model = SimpleFeedForwardNN(input_size=3, hidden_size=5, output_size=2)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Create sample data (10 samples, each with 3 features)
data = torch.randn(10, 3)
targets = torch.randn(10, 2)

# Training loop example
for epoch in range(5):
        # Forward pass
        predictions = model(data)
        loss = loss_fn(predictions, targets)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

NameError: name 'F' is not defined

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Introduction to PyTorch tensors and basic operations
print("PyTorch Fundamentals:\n")

# 1. Tensor Creation Methods
print("Different ways to create tensors:")
# From Python list
list_tensor = torch.tensor([[1, 2], [3, 4]])
# From numpy array
numpy_tensor = torch.from_numpy(np.array([[1, 2], [3, 4]]))
# Special tensors
range_tensor = torch.arange(0, 10, step=2)
linspace_tensor = torch.linspace(0, 10, steps=5)

print(f"From list:\n{list_tensor}")
print(f"\nRange tensor:\n{range_tensor}")
print(f"\nLinspace tensor:\n{linspace_tensor}")

# 2. Tensor Operations and Manipulations
print("\nTensor Operations:")
a = torch.tensor([[1, 2], [3, 4]])
b = torch.tensor([[5, 6], [7, 8]])

print(f"Matrix multiplication:\n{torch.mm(a, b)}")
print(f"\nElement-wise multiplication:\n{a * b}")
print(f"\nTranspose:\n{a.t()}")

# 3. Autograd Example
print("\nAutograd Demonstration:")
x = torch.tensor([[1., 2.]], requires_grad=True)
y = torch.tensor([[3., 4.]], requires_grad=True)
z = torch.sum(x * y)

print(f"Input x: {x}")
print(f"Input y: {y}")
print(f"z = sum(x * y): {z}")

z.backward()
print(f"\nGradient of z with respect to x: {x.grad}")
print(f"Gradient of z with respect to y: {y.grad}")

# 4. Neural Network Components
print("\nNeural Network Building Blocks:")
# Common activation functions
input_tensor = torch.randn(3)
print(f"Input: {input_tensor}")
print(f"ReLU: {F.relu(input_tensor)}")
print(f"Sigmoid: {torch.sigmoid(input_tensor)}")
print(f"Tanh: {torch.tanh(input_tensor)}")

In [ ]:
# Basic tensor operations
print("Creating tensors:")
x = torch.tensor([1, 2, 3, 4])
y = torch.tensor([5, 6, 7, 8])
print(f"x: {x}")
print(f"y: {y}")

print("\nBasic operations:")
print(f"Addition: {x + y}")
print(f"Multiplication: {x * y}")

# Creating different types of tensors
print("\nDifferent tensor types:")
zeros = torch.zeros(3, 3)
ones = torch.ones(2, 2)
random_tensor = torch.rand(2, 3)

print(f"Zeros:\n{zeros}")
print(f"\nOnes:\n{ones}")
print(f"\nRandom:\n{random_tensor}")

# Tensor properties
print("\nTensor properties:")
print(f"Shape of random tensor: {random_tensor.shape}")
print(f"Data type of x: {x.dtype}")
print(f"Device (CPU/GPU): {x.device}")

In [ ]:
 Introduction to PyTorch basics
print("PyTorch Introduction:")

# 1. Creating a simple Neural Network Layer
linear = torch.nn.Linear(in_features=2, out_features=1)
print(f"\nNeural Network Layer:\n{linear}")

# 2. Define a simple optimization problem
# Create random input data
X = torch.randn(10, 2)  # 10 samples, 2 features
y = torch.randn(10, 1)  # 10 targets

# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# 3. Simple training loop example
print("\nTraining Loop Example:")
for epoch in range(5):
    # Forward pass
    y_pred = linear(X)
    loss = criterion(y_pred, y)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 1 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

print("\nOptimization components:")
print(f"Optimizer: {type(optimizer).__name__}")
print(f"Loss Function: {type(criterion).__name__}")

<div style="width: 100%; overflow: hidden;">
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</div>